# Exercice de Machine Learning pour le cours de Data Science
## Hétic - Janvier 2019
## Cyril Bécret

In [ ]:
#Importation des librairies

import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline

## 1. Chargement des données

In [ ]:
# Load datas

TRAIN_PATH = os.path.join('..', 'input', 'train.csv')

df = pd.read_csv(TRAIN_PATH, index_col=0)
df.head()

## 2. Exploration des données

In [ ]:
# Affichage des informations sur le dataset

df.info()

Le dataset est composé de 1458644 enregistrements qui contiennent 10 colonnes.

Les colonnes sont de 4 types différents :
* INT pour ['vendor_id', 'passenger_count', 'trip_duration']
* Datetime object pour ['pickup_datetime', 'dropoff_datetime'] au format __%Y-%m-%d %H:%M:%S__.
* FLOAT pour ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'] qui correspondent à des coordonnées GPS.
* BOOLEAN pour ['store_and_fwd_flag'] stockés par 'Y' et 'N'.


In [ ]:
# Affichage des valeurs de répartition du dataset

df.describe()

## 3. Nettoyage des données

In [ ]:
# Recherche de valeurs dupliquées

df.duplicated().sum()

Nous constatons que 7 valeurs sont dupliquées dans ce dataset. Nous pouvons les retirer afin de ne pas biaiser notre analyse.

In [ ]:
# Suppression des enregistrements dupliqués et vérification du nombre total

df_no_duplicates = df.drop_duplicates()
df_no_duplicates.shape[0]

In [ ]:
# Recherche des valeurs manquantes

df_no_duplicates.isna().sum()

Aucune valeur manquante dans ce dataset. Celui-ci a été bien préparé avant d'être proposé à la compétition !

In [ ]:
# Recherche d'éventuels outliers

sns.boxplot(x=df_no_duplicates['trip_duration']).set_title("Boxplot de la durée des trajets")
plt.show();

Seule la durée des trajets présentait des valeurs extrèmes. Les autres features vérifiées ne sont donc pas montrées ici.

Nous constatons 4 valeurs qui sortent du lot pour la durée des trajets. Analysons celles-ci plus en détail :

In [ ]:
# Affichage des 4 outliers dont la durée du trajet dépasse 100000

df_no_duplicates.loc[df_no_duplicates['trip_duration'] > 100000]

Ces 4 enregistrements ont des valeurs extrèmes de durée de trajet (+27h ...). Elles seront donc retirées du dataset pour la suite de cette étude.
En regardant de plus près les dates de ces courses, nous observons qu'elles ont eu lieu en janvier et mars 2016, période pendant laquelle de violentes tempêtes de neige ont touché les USA. Ces retards sont peut être donc dû à des problèmes climatiques ? Ce n'est pas l'objet de notre étude de toute façon donc contentons nous de les retirer.
( voir : https://www.lejdd.fr/International/USA/Snowzilla-la-tempete-de-neige-fait-19-victimes-aux-Etats-Unis-769749 )

In [ ]:
# Suppression des outliers

df_clean = df_no_duplicates.loc[df_no_duplicates['trip_duration'] < 100000]
df_clean.shape[0]

## 4. Feature engineering

Nous allons créer plusieurs nouvelles features pour aider l'entrainement de notre modèle.

Commençons par la distance entre les points d'arrivés et de départ qui seront **delta_longitude** et **delta_latitude** en calculant la valeur absolue de la différence entre ces coordonnées GPS.

Ensuite, **delta_total** sera la somme de ces deux précédents deltas.

In [ ]:
# Création de features de distance entre le départ et l'arrivée pour la longitude et la latitude

df_enhanced = df_clean.copy()

df_enhanced["delta_longitude"] = np.absolute(df_clean["pickup_longitude"] - df_clean["dropoff_longitude"])
df_enhanced["delta_latitude"] = np.absolute(df_clean["pickup_latitude"] - df_clean["dropoff_latitude"])
df_enhanced["delta_total"] = df_enhanced["delta_longitude"] + df_enhanced["delta_latitude"]

Analysons ensuite les datetimes fournies dans les données de départ. Nous utiliserons le pickup_datetime puisqu'il correspond à la donnée connue dans les courses à prédire.

Nous allons extraire l'heure de la journée sous forme décimale pour y inclure les minutes.

Nous prendrons ensuite le jour de la semaine sous forme d'entier

Enfin nous regarderons le numéro de la semaine, toujours sous forme d'entier.

In [ ]:
# Création de features à partir des informations du pickup_datetime

df_enhanced["pickup_Timestamp"] =  pd.to_datetime(df_clean["pickup_datetime"], format='%Y/%m/%d')
df_enhanced["pickup_time"] = df_enhanced["pickup_Timestamp"].dt.strftime('%-H').astype(int) + (df_enhanced["pickup_Timestamp"].dt.strftime('%-M').astype(float) / 60)
df_enhanced["pickup_daynumber"] = df_enhanced["pickup_Timestamp"].dt.strftime('%w').astype(int)
df_enhanced["pickup_weeknumber"] = df_enhanced["pickup_Timestamp"].dt.strftime('%U').astype(int)

In [ ]:
# Selection des features

df_features = df_enhanced[['delta_total', 'pickup_time', 'pickup_daynumber', 'pickup_weeknumber']]
df_target = df_enhanced['trip_duration']
df_features.head()

## 5. Entraînement du modèle

In [ ]:
# Découpage du dataset en données d'entrainement (3/4) et en données de test (1/4)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_features, df_target, test_size=0.25)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# Utilisation du modèle de régression linéaire SGDRegressor

from sklearn.linear_model import SGDRegressor

regr = SGDRegressor(max_iter=1000, tol=1e-3)
regr.fit(X_train, y_train)

## 6. Prédictions

In [ ]:
# Prédictions de la durée des trajets du dataframe de test.

predicted = regr.predict(X_test)

In [ ]:
# Vérification de la précision de ces prédictions avec le RMSLE

from sklearn.metrics import mean_squared_log_error

np.sqrt(mean_squared_log_error( y_test, predicted ))

## 7. Soumission des prédictions

In [ ]:
# Importation du dataset de test

test = pd.read_csv('../input/test.csv')


In [ ]:
# Calculs des nouvelles features sur ce nouveau dataset

test["delta_longitude"] = np.absolute(test["pickup_longitude"] - test["dropoff_longitude"])
test["delta_latitude"] = np.absolute(test["pickup_latitude"] - test["dropoff_latitude"])
test["delta_total"] = test["delta_longitude"] + test["delta_latitude"]
test["pickup_Timestamp"] =  pd.to_datetime(test["pickup_datetime"], format='%Y/%m/%d')
test["pickup_time"] = test["pickup_Timestamp"].dt.strftime('%-H').astype(int) + (test["pickup_Timestamp"].dt.strftime('%-M').astype(float) / 60)
test["pickup_daynumber"] = test["pickup_Timestamp"].dt.strftime('%w').astype(int)
test["pickup_weeknumber"] = test["pickup_Timestamp"].dt.strftime('%U').astype(int)

In [ ]:
# On conserve les features que nous utilisons lors de notre prédiction

test_X = test[['delta_total', 'pickup_time', 'pickup_daynumber', 'pickup_weeknumber']]
test_X.head()

In [ ]:
# Application de notre modèle de prédiction sur le dataset de test

predicted_duration = regr.predict(test_X)
predicted_duration

In [ ]:
# Préparation de la soumission

submission = pd.DataFrame({'id': test.id, 'trip_duration': predicted_duration})

submission.to_csv('submission.csv', index=False)

